In [1]:
%pylab inline
import geopandas as gpd
import pandas as pd
import os
# !pip install mplleaflet
import mplleaflet


Populating the interactive namespace from numpy and matplotlib


In [2]:
fish_pass = 'ADFG/ADFG_fish_passage.shp'
fp = gpd.read_file(fish_pass)
regs = 'HUC8_rebuild/output/sasap_regions.shp'
regions = gpd.read_file(regs)

In [3]:
fp = fp.to_crs(epsg=3338)
regions = regions.to_crs(epsg=3338)

Each FP_color represents a fish passage impact rating (listed under RGGRating). I want to get eventually get counts of FP_color per region from regions gdf. Then map these in some way (or multiple ways).

In [4]:
fp['FP_red'] = fp['RGGRating'] == 'Red'
fp['FP_green'] = fp['RGGRating'] == 'Green'
fp['FP_gray'] = fp['RGGRating'] == 'Gray'
fp['FP_black'] = fp['RGGRating'] == 'Black'

Created a new gdf *RGG*

In [5]:
RGG = fp[['geometry','FP_red', 'FP_green', 'FP_gray', 'FP_black']]
RGG.head()

,geometry,FP_red,FP_green,FP_gray,FP_black
0,POINT (83811.68299580242 819266.9230174191),True,False,False,False
1,POINT (213872.4420121688 2252881.915418045),False,True,False,False
2,POINT (213536.4334304724 2250613.46924101),True,False,False,False
3,POINT (207221.6008933302 2235195.704555049),False,True,False,False
4,POINT (210232.879883608 2156742.824674348),True,False,False,False


Find points within each region

In [ ]:
for reg in regions.region: 
    sub_gdf = regions.query("region==@reg") 
    geom_ser = sub_gdf.geometry # here we are getting a series of geometries (geometry already part of gdf)
    geom = geom_ser.iloc[0] # here is our geometry object, iloc based on interger location based on indexing for selection by position
    RGG[reg] = RGG.within(geom)

In [7]:
RGG.head()

,geometry,FP_red,FP_green,FP_gray,FP_black,Aleutian Islands,Arctic,Bristol Bay,Chignik,Copper River,Kodiak,Kotzebue,Kuskokwim,Cook Inlet,Norton Sound,Prince William Sound,Southeast,Yukon
0,POINT (83811.68299580242 819266.9230174191),True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
1,POINT (213872.4420121688 2252881.915418045),False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
2,POINT (213536.4334304724 2250613.46924101),True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
3,POINT (207221.6008933302 2235195.704555049),False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
4,POINT (210232.879883608 2156742.824674348),True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False


I want to now get counts of each impact (colors) per region

In [11]:
# RGG has the impact for each geometry, does it have the geometry object, I imagine it does I just don't see that?
RGG['Red_Kodiak'] = np.where((RGG['FP_red'] == True) & (RGG['Kodiak'] == True), True, False)

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
RGG['Red_Kodiak'].head()

0     True
1    False
2    False
3    False
4    False
Name: Red_Kodiak, dtype: bool

In [13]:
RGG['Red_Kodiak'].sum()

67

I could continue to do this manually for each color and region but that would be a pain and inefficient. 

In [ ]:
# may need to leave RGG as a series? 
# maybe I need to iterate over multiple columns?
for row, col in RGG.iterrows(): # maybe categorize these by geometry within this gdf? then move onto regions
    FP_red, FP_green, FP_black, FP_gray = col
    for r, c in # there are 13? regions seems like a lot to type